# Shingling and Jaccard's similarity

Practical course material for the ASDM Class 09 (Text Mining) by Florian Leitner.

© 2017 Florian Leitner. All rights reserved.

## Shingling: generating n-grams

### Definition

In [1]:
def shingle(s, k, joiner='_'):
    """Generate `k`-sized n-grams of a string or list of strings `s`."""
    k = min(len(s), k)
    merge = lambda i: i
    
    if joiner is not None and isinstance(s, list):
        merge = lambda i: joiner.join(i)
    
    for i in range(len(s) - k + 1):
        yield merge(s[i:i+k])

### Examples

Create all character trigrams of a string:

In [2]:
list(shingle('string', 3))

['str', 'tri', 'rin', 'ing']

Create all word trigrams of a tokenized sentence:

In [3]:
list(shingle(['This', 'is', 'a', 'tokenized', 'sentence', '.'], 3))

['This_is_a', 'is_a_tokenized', 'a_tokenized_sentence', 'tokenized_sentence_.']

If you want to avoid joining the higher n-grams of a token sequence:

In [4]:
list(shingle(['This', 'is', 'a', 'tokenized', 'sentence', '.'], 3, None))

[['This', 'is', 'a'],
 ['is', 'a', 'tokenized'],
 ['a', 'tokenized', 'sentence'],
 ['tokenized', 'sentence', '.']]

### Shingle output is not unique

In [5]:
seq = list(shingle('amabama', 2))
print("len(", type(seq), ") =", len(seq))

len( <class 'list'> ) = 6


In [6]:
seq = set(shingle('amabama', 2))
print("len(", type(seq), ") =", len(seq))

len( <class 'set'> ) = 4


## Jaccard's similarity

### Definition

In [7]:
def jaccard(seq1, seq2):
    """The Jaccard similarity between two sequences: |∩(X,Y)| / |∪(X,Y)|."""
    x = set(seq1)
    y = set(seq2)
    # Python3 does int->float casting on integer divisions
    # See "Banker's rounding, probabilies, and underflows"
    return len(x & y) / len(x | y)

### Examples

In [8]:
jaccard('string', 'string')

1.0

In [9]:
jaccard('string', 'strang')

0.7142857142857143

In [10]:
jaccard('string', 'other')

0.2222222222222222

### Issues

In [11]:
jaccard('string', 'other') # too large?

0.2222222222222222

In [12]:
jaccard('karlos', 'carol') # ok-ish, but high?

0.5714285714285714

In [13]:
jaccard('word', 'ford') # great

0.6

In [14]:
jaccard('alabama', 'malba') # really bad!

1.0

Therefore, it typically is better to use higher order; **Bigram** similarity:

In [15]:
jaccard(shingle('string', 2), shingle('other', 2)) # great

0.0

In [16]:
jaccard(shingle('karlos', 2), shingle('carol', 2)) # too small?

0.125

In [17]:
jaccard(shingle('word', 2), shingle('ford', 2)) # a bit too small?

0.5

In [18]:
jaccard(shingle('alabama', 2), shingle('malba', 2)) # still too large?

0.42857142857142855

Or even character **trigrams**:

In [19]:
jaccard(shingle('string', 3), shingle('other', 3)) # great

0.0

In [20]:
jaccard(shingle('karlos', 3), shingle('carol', 3)) # really bad!

0.0

In [21]:
jaccard(shingle('word', 3), shingle('fort', 3)) # really bad!

0.0

In [22]:
jaccard(shingle('alabama', 3), shingle('malba', 3)) # great

0.0

### Multigram Jaccard similarity

In [23]:
def multigram(seq):
    return list(seq) + list(shingle(seq, 2))
                                                   
print(sorted(multigram('string')))
print(sorted(multigram(['This', 'is', 'a', 'tokenized', 'sentence', '.'])))

['g', 'i', 'in', 'n', 'ng', 'r', 'ri', 's', 'st', 't', 'tr']
['.', 'This', 'This_is', 'a', 'a_tokenized', 'is', 'is_a', 'sentence', 'sentence_.', 'tokenized', 'tokenized_sentence']


In [24]:
jaccard(multigram('karlos'), multigram('carol')) # reasonable...

0.3333333333333333

In [25]:
jaccard(multigram('word'), multigram('ford')) # reasonable...

0.5555555555555556

In [26]:
jaccard(multigram('alabama'), multigram('malba')) # reasonable...

0.6363636363636364

In [27]:
jaccard(multigram('string'), multigram('other')) # resasonable...

0.1111111111111111

In [28]:
jaccard(multigram('string'), multigram('string')) # of course (but do check!)

1.0

In [29]:
jaccard(multigram('string'), multigram('strang')) # reasonable...

0.5714285714285714

### Conclusion

In the end, depending on the scenario, a problem-specific decision for using uni-, bi- or, maybe a trigram sets has to be made.